## Goal: write an article about prediction intervals and a nice interface for sklearn


### Process:

#### Let's see if I can do it with some sort of data already in sklearn without internet? 

YES I CAN.

#### Can I explore sampling error?

YES I CAN.

#### How would I use it? 

I'd need to basically always test the sample inside a modeling workflow. Want to return a p value + user defined (default 95%) confidence interval. Let's do it by simulation.

1) Do a train test split  
2) On training set, resample with replacement 1000 times on all or 1000 points  
3) Calculate p value and confidence interval and return them  

#### Can I explore modeling error?

To explore the modeling error I'd get the threshold, the z score based on the confidence level, and the standard deviation.

1) Convert threshold into z score
2) Multiply times std deviation
3) move off of prediction

All predicted points should have the same error right?



In [1]:
import sklearn
import pandas as pd
import numpy as np
from sklearn import datasets
from scipy import stats

## A little info on the dataset

In [2]:
print(datasets.load_boston()['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

# Preprocess data

In [3]:
df = pd.DataFrame(datasets.load_boston()['data'])
df.columns = datasets.load_boston()['feature_names']
df['target'] = datasets.load_boston()['target']
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# Explore sample error

In [4]:
simulations_40 = []
for i in range(1000):
    simulations_40.append(df.sample(40).target.mean())
    
simulations_5 = []
for i in range(1000):
    simulations_5.append(df.sample(5).target.mean())
    
upper_40 = np.percentile(simulations_40,97.5)
lower_40 = np.percentile(simulations_40,2.5)

upper_5 = np.percentile(simulations_5,97.5)
lower_5 = np.percentile(simulations_5,2.5)

print(upper_40,upper_5,lower_40,lower_5,df.target.mean())

25.2304375 31.283999999999995 19.774812499999996 15.539500000000002 22.532806324110698


# Build a little sklearn extension

1) Do a train test split  
2) On training set, resample with replacement 1000 times on all or 1000 points  
3) Calculate p value and confidence interval and return them

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target',axis=1), df.target, test_size=0.2, random_state=42)

In [7]:
def target_confidence_interval(target, ci=.95, num_simulations=1000):
    '''
    See if the mean of your target in your training set is 
    statistically significant and return the confidence interval for it.
    
    INPUTS: 
        - A target variable in a pandas series dtype (or other object with a matching sample method) 
        - A level of confidence you want (default is .95)
        - Number of times you want to run simulation (1000 is default)
    RETURNS:
        - p-value #STILL NEED TO IMPLEMENT
        - lower and upper bounds of confidence interval
    '''

    #simulate resampling by sampling with replacement n times
    #to generate distribution of mean target series
    simulation_results = []
    for i in range(num_simulations):
        simulation_results.append(
            target.sample(target.shape[0], replace=True).mean())

    #get the desired percentiles
    inverse_ci = 1 - ci
    lower_percentile = (inverse_ci / 2) * 100
    upper_percentile = (1 - (inverse_ci / 2)) * 100

    #still need to add something to calculate p-value in a line below and then return it

    #calculate upper and lower ci bounds
    lower = np.percentile(simulation_results, lower_percentile)
    upper = np.percentile(simulation_results, upper_percentile)
    
    ttest, pval = stats.ttest_ind(target, df.target) 

    return lower, upper, pval

I'm not sure if this code does anything useful.

# Look at modeling prediction intervals

In [15]:
import seaborn as sns
import statsmodels.api as sm
model = sm.OLS(y_train,X_train)
fit_model = model.fit()

In [9]:
predictions = fit_model.predict(X_test)

In [11]:
sum_errs = np.sum((y_test - predictions)**2)
stdev = np.sqrt(1/(len(y_test)-2) * sum_errs)
interval = 1.96 * stdev
lower, upper = predictions - interval, predictions + interval

In [12]:
d = {'avg_rooms':X_test.RM,
     'lower':lower,
    'predictions':predictions,
    'upper':upper}
res_df = pd.DataFrame(d)

In [22]:
styles = ['b--','r-','b--']
#df.plot(x='crime_rate', y=['lower','predictions','upper'], style=styles)
res_df.plot(x='avg_rooms', y='predictions')

In [31]:
#not sure why this code starts putting graphs in a separate window
%pylab
plt.scatter(res_df.avg_rooms, res_df.predictions)
x_in = X_test.RM.values[0]
yhat_out = predictions[0]
#plt.plot(X_test.RM, predictions, color='red')
plt.errorbar(x_in, yhat_out, yerr=interval, color='black', fmt='o')
plt.title('Example prediction interval for 1 prediction in a regression')
plt.xlabel('Average number of rooms')
plt.ylabel('Median house price (in thousands)')

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


Text(0, 0.5, 'Median house price (in thousands)')

In [29]:
sns.lineplot(x='avg_rooms',y='lower',data=res_df)
sns.lineplot(x='avg_rooms',y='predictions',data=res_df)
sns.lineplot(x='avg_rooms',y='upper',data=res_df)
plt.ylabel('predictions');



/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Text(0, 0.5, 'predictions')